## Наивный метод интеграции графа знаний

Наивный подход для построения RAG на основе графа знаний подразумевает поиск по узлам [21]. Опишем модули, которые отличаются своей реализации от базового подхода построения вопросно-ответных систем описанный в 2.2. 
- **Модуль индексации:** и подготовки базы знаний: В данном модуле предлагается проводить построение графа с помощью извлечения структуры документа docx, как описано в параграфе 5.2.  Для каждого узла графа типа чанк или параграф найдем эмбеддинги текстов, содержащихся в этих узлах. 


- **Модуль поиска**: Алгоритм поиска остается таким же, как и в пункте 3.2. Только теперь, каждый документ — это вершина графа типа чанк или параграф. Вершины графа также описаны в пункте 3.2.


- **Модуль анализа результатов поиска**: после выполнения поиска у нас есть список вершин графа, которые потенциально могут содержать в себе ответ на запрос. После чего происходи поиск дополнительных данных используя структуру графа – находятся все близкие вершины к найденной вершине. Для поиска близких вершин воспользуемся языком запросов Cypher - Листинг 5.
Листинг 5. Cypher запрос для поиска соседей вершины. 
```
MATCH path = (current)-[r*1..6]-(neighbor:Chunk_node)
WHERE current.id = '{meta['id']}'
AND NONE(rel IN relationships(path) WHERE type(rel) IN ['MENTION', 'MENTIONS', 'PP_CONTAINS'])
WITH neighbor, length(path) AS distance
ORDER BY distance
RETURN neighbor
```

### подготовка базы знаний

In [14]:
# run examples/graph_creation/graph_splitter_neo4j_base.py
# db sandbox will be saved in neo4j
db_name = 'sandbox'

In [15]:
import os
# setting for langsmith
os.environ["LANGCHAIN_PROJECT"] = "graph-rag"
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_PROJECT"] = "graph-rag"
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_API_KEY"] = "<>"

# setting for openai
os.environ["OPENAI_API_KEY"] = "<>"
os.environ["OPENAI_BASE_URL"] = 'https://api.proxyapi.ru/openai/v1'

# setting for neo4j
os.environ["NEO4J_URI"] ="bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "login"
os.environ["NEO4J_PASSWORD"] = "pass"


In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings

## индексируем документы 

In [17]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Chunk_node",
    text_node_properties=["text", 'id'],
    embedding_node_property="embedding",
    database=db_name,
    index_name='main_index'
)

In [18]:
# test
search_engine = vector_index.similarity_search_with_score("скан-архив", k=5)

## загружаем LLM

In [19]:
from langchain_openai import ChatOpenAI
llm_gpt = ChatOpenAI(base_url='https://api.proxyapi.ru/openai/v1')

## Загружаем граф

In [20]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(database=db_name)
retriever = vector_index.as_retriever()

## Собираем RAG

In [21]:
import re

def parse_to_dict(input_str):
    # Создаем пустой словарь для хранения результатов
    result_dict = {}
    
    pattern = r'\n(text|id):\s*([^\n]+)'
    entries = re.findall(pattern, input_str)

    # Заполняем словарь найденными парами ключ-значение
    for key, value in entries:
        result_dict[key.strip()] = value.strip()
    
    return result_dict

In [22]:
prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении заданий по поиску ответов на вопросы. Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. Если вы не знаете ответа, просто скажите, что вы не знаете. 
           Вопрос: {question} 
           Контекст: {context} 
           Ответ:""")

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    context = []
    for doc in docs:
        meta = parse_to_dict(doc.page_content)
        context.append(meta['text'])
        
        # тепрь найдем соседей на 3 близкие чанки к текущему
        neighbord = graph.query(f"""
                                MATCH path = (current)-[r*1..4]-(neighbor:Chunk_node)
                                WHERE current.id = '{meta['id']}'
                                AND NONE(rel IN relationships(path) WHERE type(rel) IN ['MENTION', 'MENTIONS', 'PP_CONTAINS'])
                                WITH neighbor, length(path) AS distance
                                ORDER BY distance
                                RETURN neighbor                                    
                                """)
        
        for ne in neighbord[:3]:
            c_t = ne['neighbor']['text'] 
            context.append(c_t)
    
    print("\n\n".join(context))
        
    return "\n\n".join(context)[:10000]

In [24]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_gpt
    | StrOutputParser()
)

In [25]:
rag_chain.invoke('как установить дозор?')

Данный документ описывает порядок подготовки к работе и эксплуатации программного продукта «Дозор. Мониторинг систем безопасности рабочей станции» (далее – ПП).Инструкция относится к релизу ПП: 1.0.0.

Мониторинг систем безопасности рабочей станции.Руководство по установкеРостов-на-Дону2022

как установить пароль для входа в программу

Как восстановить данные, занесенные в архив.Порядок действий по восстановлению данных (раскрытию архива) описан в разделе «Раскрытие архива» настоящего Руководства.Как вернуть те данные, которые были в каталоге до раскрытия архива.Перед восстановлением данных из архива происходит создание так называемого резервного архива. Он создается в соответствии с настройками этого же раздела и, таким образом, содержит те файлы, которые будут перезатерты при раскрытии другого архива. Чтобы вернуть те данные, которые были до раскрытия архива, просто раскройте соответствующий резервный архив.Как просмотреть содержимое архива.Для того чтобы просмотреть список файлов, п

'Для установки программы "Дозор. Мониторинг систем безопасности рабочей станции" следует выполнить следующие шаги:\n1. Проверить комплектность программы.\n2. Убедиться, что компьютер соответствует требованиям системы.\n3. Установить программу на компьютер.\n4. Зарегистрировать программу.\n\nЕсли вам нужно установить пароль для входа в программу, вам следует:\n1. Запустить Менеджер Хранителя.\n2. В дереве разделов перейти на корневой элемент, он имеет заголовок "Admin".\n3. Выбрать команду "Параметры" из меню "Разделы".\n4. Ввести пароль в поля "Пароль" и "Подтверждение пароля".\n5. Закрыть диалог (кнопка "OK").\n\nЭти действия помогут вам установить пароль для доступа к программе.'

## Считаем метрики 

### собираем ответы на тест

In [ ]:
import pandas as pd
test_set = pd.read_csv('../../assets/test_set_40.csv')

test_set['answer'] = test_set['question'].apply(lambda x: rag_chain.invoke(x.strip()))
test_set['contexts'] = test_set['contexts'].apply(lambda x:[x])
test_set.to_csv("naive_method.csv", index=False)

### собираем метрики 

In [27]:
import datasets
test_dataset = datasets.Dataset.from_dict(test_set)
from ragas.metrics import (
    answer_relevancy,
    context_relevancy,
    answer_relevancy,
    answer_similarity
)
from ragas import evaluate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# ragas метрики
result = evaluate(
    test_dataset,
    metrics=[
        context_relevancy,
        answer_relevancy,
        answer_similarity,
    ],
    llm=ChatOpenAI(model_name="gpt-3.5-turbo-0125"),
    embeddings=OpenAIEmbeddings()
)

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

In [2]:
from src.metrics.metric_zoo import calculate_cosine_similarity_TF_IDF, calculate_similarity_spacy, calculate_bleu

In [30]:
test_set['bleu_score'] = test_set.apply(calculate_bleu, axis=1)
test_set['sim-spacy'] = test_set.apply(calculate_similarity_spacy, axis=1)
test_set['cos-sim-TF-IDF'] = test_set.apply(calculate_cosine_similarity_TF_IDF, axis=1)

result['bleu_score'] = test_set['bleu_score'].mean()
result['sim-spacy'] = test_set['sim-spacy'].mean()
result['cos-sim-TF-IDF'] = test_set['cos-sim-TF-IDF'].mean()

/Users/kodin/Library/Caches/pypoetry/virtualenvs/kg-llm-A_Fd_ayW-py3.9/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/kodin/Library/Caches/pypoetry/virtualenvs/kg-llm-A_Fd_ayW-py3.9/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/kodin/Library/Caches/pypoetry/virtualenvs/kg-llm-A_Fd_ayW-py3.9/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore th

In [ ]:
result